# Generate today's English word

In [26]:
import json

from tqdm.notebook import tqdm
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import HumanMessage, AIMessage
import pandas as pd
import os
os.environ["OPENAI_API_KEY"] = "Your API Key"


In [48]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=1.0)

In [49]:
class Word(BaseModel):
    word: str = Field("English word")
    emoji: str = Field("appropriate emoji for the word")
    meaning: str = Field("The meaing of the word in English")

In [50]:
parser = JsonOutputParser(pydantic_object=Word)

In [51]:
format_instruction = parser.get_format_instructions()

In [52]:
fewshot = [
    HumanMessage(content=f"Please create an English word along with a suitable emoji and its meaning, following the given format.\n{format_instruction}"),
    AIMessage(content='{"word": "situation", "emoji": "🤔", "meaning": "Situation refers to a specific place or time where something is happening. It can also indicate the current state or condition of something. Situation can be used to emphasize circumstances where problems or difficulties have arisen."}'),
    HumanMessage(content=f"Please create an English word along with a suitable emoji and its meaning, following the given format.\n{format_instruction}"),
    AIMessage(content='{"word": "concern", "emoji": "😟", "meaning": "Concern means worry or anxiety. It can represent interest or worry about something that can emotionally or physically affect someone. Concern can also mean importance or relevance."}'),
    HumanMessage(content=f"Please create an English word along with a suitable emoji and its meaning, following the given format. Make sure that the word created has not created before\n{format_instruction}"),
]

In [53]:
chain = model | parser

In [54]:
chain.invoke(fewshot)

{'word': 'serendipity',
 'emoji': '🎉',
 'meaning': 'Serendipity is finding something valuable or delightful when you least expect it. It refers to a happy accident or pleasant surprise that comes out of coincidence or chance. Serendipity can lead to unexpected and positive outcomes.'}

In [55]:
sample_list = []
n_sample = 10
for _ in tqdm(range(10), total=n_sample):
    sample = chain.invoke(fewshot)
    sample_list.append(sample)
    

  0%|          | 0/10 [00:00<?, ?it/s]

In [57]:
df = pd.DataFrame(sample_list)

In [58]:
df

,word,emoji,meaning
0,serendipity,🍀,Serendipity is the occurrence of events by cha...
1,serendipity,🍀,Serendipity refers to the occurrence of events...
2,serendipity,🍀,Serendipity refers to the occurrence of events...
3,Whimsical,🤪,"Whimsical describes something that is playful,..."
4,serendipity,🍀,Serendipity refers to the occurrence of fortun...
5,serendipity,🌈,Serendipity refers to the occurrence of events...
6,serendipity,🍀,Serendipity is the occurrence of events by cha...
7,serendipity,🍀,Serendipity is the occurrence of events by cha...
8,serendipity,🍀,Serendipity is the occurrence of events by cha...
9,Serendipity,🍀,Serendipity is the occurrence of events by cha...


In [60]:
df.to_excel("./words.xlsx", index=False)